<a href="https://colab.research.google.com/github/Kulkarni-mohit/Fine-tuning-sentiment-analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Useful libraries

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

## Sentiment Analysis using pipeline()

In [2]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
classifier = pipeline('sentiment-analysis',model = model, tokenizer = tokenizer)

In [5]:
results = classifier(["The Movie was awesome.", "The action of hero was not that good."])

In [6]:
for i in results:
  print(i)

{'label': 'POSITIVE', 'score': 0.9998719692230225}
{'label': 'NEGATIVE', 'score': 0.999768078327179}


## Sentiment Analysis using Pytorch

In [7]:
tokens = tokenizer.tokenize("The Movie was awesome!!")
token_id = tokenizer.convert_tokens_to_ids(tokens)
input_id = tokenizer("The Movie was awesome!!")

In [8]:
print(f'Tokens: {tokens}')
print(f'Token IDs : {token_id}')
print(f'Input IDs : {input_id}')

Tokens: ['the', 'movie', 'was', 'awesome', '!', '!']
Token IDs : [1996, 3185, 2001, 12476, 999, 999]
Input IDs : {'input_ids': [101, 1996, 3185, 2001, 12476, 999, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [9]:
X_train = ["The Movie was awesome.", "The action of hero was not that good."]

In [10]:
batch = tokenizer(X_train, padding = True, truncation = True, max_length=512, return_tensors='pt')

In [11]:
batch

{'input_ids': tensor([[  101,  1996,  3185,  2001, 12476,  1012,   102,     0,     0,     0,
             0],
        [  101,  1996,  2895,  1997,  5394,  2001,  2025,  2008,  2204,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
# with torch.no_grad():
outputs = model(**batch, labels=torch.tensor([1,0]))
print(outputs)
predictions = F.softmax(outputs.logits, dim=1)
print(predictions)
labels = torch.argmax(predictions, dim=1)
print(labels)
labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
print(labels)

SequenceClassifierOutput(loss=tensor(0.0002, grad_fn=<NllLossBackward0>), logits=tensor([[-4.2912,  4.6718],
        [ 4.6390, -3.7297]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[1.2804e-04, 9.9987e-01],
        [9.9977e-01, 2.3194e-04]], grad_fn=<SoftmaxBackward0>)
tensor([1, 0])
['POSITIVE', 'NEGATIVE']
